# 本題要找出有哪些是忠實的客戶 (\#GroupBy, \#Join, \#DateDiff )
原題目連結：https://leetcode.com/problems/find-loyal-customers/description/  

Table: `customer_transactions`.   
```
+------------------+---------+
| Column Name      | Type    | 
+------------------+---------+
| transaction_id   | int     |
| customer_id      | int     |
| transaction_date | date    |
| amount           | decimal |
| transaction_type | varchar |
+------------------+---------+
```
transaction_id is the unique identifier for this table.  
transaction_type can be either 'purchase' or 'refund'.  
Write a solution to find loyal customers. A customer is considered loyal if they meet ALL the following criteria:  

Made at least 3 purchase transactions.  
Have been active for at least 30 days.  
Their refund rate is less than 20% .  
Return the result table ordered by customer_id in ascending order.  

The result format is in the following example.  

範例：  
Ex1:  
Input:  

`customer_transactions` table:  
```
+----------------+-------------+------------------+--------+------------------+
| transaction_id | customer_id | transaction_date | amount | transaction_type |
+----------------+-------------+------------------+--------+------------------+
| 1              | 101         | 2024-01-05       | 150.00 | purchase         |
| 2              | 101         | 2024-01-15       | 200.00 | purchase         |
| 3              | 101         | 2024-02-10       | 180.00 | purchase         |
| 4              | 101         | 2024-02-20       | 250.00 | purchase         |
| 5              | 102         | 2024-01-10       | 100.00 | purchase         |
| 6              | 102         | 2024-01-12       | 120.00 | purchase         |
| 7              | 102         | 2024-01-15       | 80.00  | refund           |
| 8              | 102         | 2024-01-18       | 90.00  | refund           |
| 9              | 102         | 2024-02-15       | 130.00 | purchase         |
| 10             | 103         | 2024-01-01       | 500.00 | purchase         |
| 11             | 103         | 2024-01-02       | 450.00 | purchase         |
| 12             | 103         | 2024-01-03       | 400.00 | purchase         |
| 13             | 104         | 2024-01-01       | 200.00 | purchase         |
| 14             | 104         | 2024-02-01       | 250.00 | purchase         |
| 15             | 104         | 2024-02-15       | 300.00 | purchase         |
| 16             | 104         | 2024-03-01       | 350.00 | purchase         |
| 17             | 104         | 2024-03-10       | 280.00 | purchase         |
| 18             | 104         | 2024-03-15       | 100.00 | refund           |
+----------------+-------------+------------------+--------+------------------+
```
Output:  
```
+-------------+
| customer_id |
+-------------+
| 101         |
| 104         |
+-------------+
```
Explanation:  

Customer 101:  
Purchase transactions: 4 (IDs: 1, 2, 3, 4)   
Refund transactions: 0  
Refund rate: 0/4 = 0% (less than 20%)   
Active period: Jan 5 to Feb 20 = 46 days (at least 30 days)   
Qualifies as loyal   
Customer 102:  
Purchase transactions: 3 (IDs: 5, 6, 9)   
Refund transactions: 2 (IDs: 7, 8)  
Refund rate: 2/5 = 40% (exceeds 20%)   
Not loyal   
Customer 103:  
Purchase transactions: 3 (IDs: 10, 11, 12)   
Refund transactions: 0  
Refund rate: 0/3 = 0% (less than 20%)   
Active period: Jan 1 to Jan 3 = 2 days (less than 30 days)   
Not loyal   
Customer 104:  
Purchase transactions: 5 (IDs: 13, 14, 15, 16, 17)   
Refund transactions: 1 (ID: 18)  
Refund rate: 1/6 = 16.67% (less than 20%)   
Active period: Jan 1 to Mar 15 = 73 days (at least 30 days)   
Qualifies as loyal   
The result table is ordered by customer_id in ascending order.  

* 解題想法：  
首先用row_number()找出每個customer_id有的資料，並找出最新與最舊的活動資料來找出天數，接著用groupby找出購買以及退貨的筆數，並算出退貨的比例，最後將兩個條件join之後找出符合條件的customer_id就是答案

```
# Write your MySQL query statement below
select a.customer_id from 
(
select customer_id from (
select a.customer_id, datediff(b.transaction_date, a.transaction_date) as d from 
(
select customer_id, transaction_date from 
(
    select row_number() over (partition by customer_id order by transaction_date) as id, customer_id, transaction_date from customer_transactions
) c where id = 1
) a left join 
( 
select customer_id, transaction_date from 
(
    select row_number() over (partition by customer_id order by transaction_date desc) as id, customer_id, transaction_date from customer_transactions
) c where id = 1
) b 
on a.customer_id = b.customer_id
) days where d >= 30
) a 
left join 
(
    select a.customer_id, times, p, ifnull(r, 0) as r, p + ifnull(r, 0) as t from 
    (
        select customer_id, count(*) as times from customer_transactions group by customer_id
    ) a
    left join 
    (
        select customer_id, count(amount) as p from customer_transactions where transaction_type = 'purchase' group by customer_id
    ) b on a.customer_id = b.customer_id 
    left join 
    (
        select customer_id, count(amount) as r from customer_transactions where transaction_type = 'refund' group by customer_id
    ) c on a.customer_id = c.customer_id
    where times >= 3
) b
on a.customer_id = b.customer_id
where r / t < 0.2 order by customer_id
```